# Tick 3

* [Pandas warmup exercises](#notes3) &mdash; not assessed
* [Tick 3a](#tick3a) &mdash; worth 1 mark
* [Tick 3b](#tick3b) &mdash; worth 1 mark
* [Tick 3*](#tick3star) &mdash; not assessed

# Pandas warmup exercises (not assessed)<span id="notes3"></span>

<div class="alert alert-warning">
    These are optional warmup exercises, to get you used to pandas. 
    </div>

Use the following autograder settings:
```
import ucamcl
GRADER = ucamcl.autograder('https://markmy.solutions', course='scicomp').subsection('notes3')
```
Each question has a label (in brackets). Call `fetch_question` to see a model answer.
```
q = GRADER.fetch_question('ex1')
# view the answer
print(q)
```

**Exercise (ex1)** from section 3.5. In pandas, to find the most frequent value (the _mode_)
of `age_range` in the `stopsearch` dataset, we can use
```
stopsearch['age_range'].mode().values[0]
```
(The `mode()` call returns an object, and `.values[0]` extracts just the value itself.)

How would you generate a table showing the most frequent age range for each combination of ethnicity and gender?

**Exercise (ex2)** from section 3.4 and 3.5. 
Given the dataframe
```
df = pandas.DataFrame({'A': [0,0,0,1,1,1], 'B': [0,1,2,0,1,2], 'X': range(6)})
```
how do you produce a table that has rows for A, columns for B, and shows values of X?

**Exercise (ex3)** from section 3.6. This code produces a dataframe with columns for ethnicity, outcome, and n:
```
sscam = stopsearch.loc[stopsearch.force=='cambridgeshire'].copy()
sscam['outcome'] = np.where(sscam.outcome == 'False', 'nothing', 'find')
df = sscam.groupby(['officer_defined_ethnicity','outcome']).apply(len).reset_index(name='n')
```
Take the subtable with `outcome=='nothing'`, and the subtable with `outcome='find'`, and merge them; then use the merged table to compute the
`percent_find` column from section 3.6 of notes. 

**Exercise (ex4)** from section 3.5. As an alternative to the method in exercise ex3, 
take the indexed array
```
sscam.groupby(['officer_defined_ethnicity','outcome']).apply(len)
```
and convert it to a wide-form dataframe, then use this to compute `percent_find`.

# Tick 3a. Analysis of flood data<span id="tick3a"></span>

This section is worth 1 mark. Use these autograder settings:
```
import ucamcl
GRADER = ucamcl.autograder('https://markmy.solutions', course='scicomp').subsection('tick3a')
```

<div class="alert alert-warning">
This assignment tests your skill at manipulating dataframes and indexed arrays.
    <strong>YOUR CODE MUST USE PANDAS AND NUMPY OPERATIONS FOR DATA MANIPULATION</strong>
    rather than for loops, wherever possible.
</div>

## Introduction

![Jesus Green lock](res/jesus_green_lock.jpg)

This assignment asks you to analyse data provided by the UK Environment Agency concerning flooding. The agency offers an [API for near real-time data](http://environment.data.gov.uk/flood-monitoring/doc/reference) covering:
* flood warnings and flood alerts
* flood areas which to which warnings or alerts apply
* measurements of water levels and flows
* information on the monitoring stations providing those measurements

In this assignment we will be working with historical data of water level measurements, at several monitoring stations in Cambridge and on the Cam. The dataset is available as a CSV file at [https://teachingfiles.blob.core.windows.net/datasets/flood_20191125.csv](https://teachingfiles.blob.core.windows.net/datasets/flood_20191125.csv).

_Image by [N. Chadwick](http://www.geograph.org.uk/photo/4800494)._

## Questions

**Question 1.** Import the CSV file and print out a few lines, choosing the lines at random using `np.random.choice`. The file mistakenly includes records from a River Cam in Gloucestershire, and another River Cam in Somerset. Remove these rows, and store what's left as the data frame `flood`. How many rows are left?
```
# Submit your answer:
GRADER.submit_answer(GRADER.fetch_question('q1'), num_rows)
```

_Hint. [`DataFrame.drop_duplicates`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html) may be useful for seeing what's in the file._

**Question 2.** Complete this table of the number of entries in this dataset for each town and river.

| . | Cambridge | Great Chesterford | Great Shelford | Milton |
| -- |||||
| **Bin Brook** | 2628 | 0 |||
| **River Cam** |||||

```
# Submit your answer
your_ans = ...  # an unstacked indexed array
GRADER.submit_answer(GRADER.fetch_question('q2'), your_ans.values)
```

**Question 3.** Each water measuring station has a distinct `measure_id` and `label`. Complete this dataframe of the number of measurement stations for each town and river. Use only the Pandas operations for split-apply-combine, don't use any numpy operations or Python `for` loops or list comprehensions. 

| town | num_stations |
|--||
| Milton | 1 |
| Great Shelford | 1 |

```
# Submit your answer. Row order doesn't matter.
GRADER.submit_answer(GRADER.fetch_question('q3'), your_dataframe)
```

**Question 4.** 
Each measurement station has low and high reference levels, in columns `low` and `high`. In this dataset, the reference levels are stored for every measurement, but we can verify that every `measure_id` has a unique pair `(low,high)` with
```
assert all(flood.groupby(['measure_id','low','high']).apply(len).groupby('measure_id').apply(len) == 1), "Reference levels non-constant"
```
Add a column `norm_value`, by rescaling `value` linearly so that `value=low` correponds to `norm_value=0` and `value=high` corresponds to `norm_value=1`.
Use `np.nanquantile` to find the [_tercile points_](https://en.wiktionary.org/wiki/tercile), the two values that split the entire `norm_value` column into three roughly equal parts.

```
# Submit your answer:
GRADER.submit_answer(GRADER.fetch_question('q4'), [tercile1, tercile2])
```

**Question 5.** Complete the following dataframe, listing the number of observations in each tercile and the total. (When there are repeated values, it's arbitrary how we assign observations into terciles. To answer this question, use the convention that `med` means `tercile1 <= value < tercile2`.)

| label | norm_value_tercile | n | ntot |
|--||||
| Bin Brook | med | 2466 | 2628 |
| Bin Brook | high | 162 | 2628 |
| Cambridge Baits Bite | low | 2 | 2813 |

```
# Submit your answer. Row order doesn't matter.
GRADER.submit_answer(GRADER.fetch_question('q5'), your_dataframe)
```

**Question 6.** Complete this dataframe, listing the fraction of observations in each tercile per station:

| label | low | med | high |
|---- ||||
| Bin Brook | 0.000 | 0.938 | 0.062 |
| Cambridge Jesus Lock | 0.032 | 0.356 | 0.612 |

```
# Submit your answer. Row order doesn't matter. Don't round.
GRADER.submit_answer(GRADER.fetch_question('q6'), your_dataframe)
```

**Question 7.** Fill in the rest of this indexed array, giving the `low` and `high` values for each measurement station.

| label | ref | |
|----|||
| Bin Brook | high | 1.000 |
| | low | 0.057 |
| Cambridge Baits Bite | high | 0.294 |
| | low | 0.218 |

```
# Submit your answer. Let your_ans be an indexed array.
GRADER.submit_answer(GRADER.fetch_question('q7'), your_ans.reset_index(name='val'))
```
 _Hint. There are many ways to approach this. The cleanest is to use [`melt`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.melt.html#pandas.DataFrame.melt)_

# Tick 3b. Plotting

This section is worth 1 mark. There is no automated testing of your answers here, but your code may be assessed in the ticking session.

<div class="alert alert-warning">For this assignment you will need
    to figure out matplotlib's myriad options.
    You will need to spend time <a href="https://stackoverflow.com/questions/tagged/matplotlib">searching</a> for help.  
    You don't need to be pixel perfect, but you do need to demonstrate that you can control
    <ul>
        <li>aspect ratio</li>
        <li>subplots layout and spacing</li>
        <li>x and y axis ranges, and colour palette</li>
        <li>display of tick labels and gridlines</li>
        <li>legend placement</li>
        <li>titles</li>
    </ul>
</div>

**Question 8.** Reproduce this plot:

<img src="res/tick3b_bar.svg" style="height:22em"/>


**Question 9.** Reproduce this plot:

<img src="res/tick3b_timeseries.svg" style="height:30em"/>


The light shaded area shows the range from `low` to `high` for each station. The dark shaded area shows the inter-tercile range, `low+tercile1*(high-low)` to `low+tercile2*(high-low)` where `tercile1` and `tercile2` are your answers to Question 4. They can be plotted with [`ax.axhspan`](https://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.axhspan.html#matplotlib.axes.Axes.axhspan).
Here are some code snippets that may be useful, for formatting tick labels with dates.
```
import matplotlib
# Date-axis control, taken from http://matplotlib.org/examples/api/date_demo.html
ax.xaxis.set_major_locator(matplotlib.dates.WeekdayLocator(byweekday=matplotlib.dates.MO, tz=pytz.UTC))
ax.xaxis.set_minor_locator(matplotlib.dates.DayLocator(tz=pytz.UTC))
ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%a %d %b'))
```

# Tick 3* (not assessed)<span id="tick3star"></span>

**Question 10.** The real story in this dataset is about the [Yorkshire floods of 2019](https://en.wikipedia.org/wiki/2019_Yorkshire_floods). Using the full dataset of all readings, 
plot a map showing the peak water level at each measuring station. Here's an illustration.

<img src="res/tick3star_flooding.svg" style="height:20em"/>

You can download the full dataset in three files,

* https://teachingfiles.blob.core.windows.net/datasets/stations_20191125.csv
* https://teachingfiles.blob.core.windows.net/datasets/measures_20191125.csv
* https://teachingfiles.blob.core.windows.net/datasets/readings_20191125.csv

You will have to merge these files: `readings.measure_id` refers to `measures.measure_id`, and `measures.station_uri` refers to `station.uri`. You will also have to filter the readings, and only keep those with `parameter=='Water Level'`.

**Question 11.** The dataset also includes rainfall measurements. Plot rainfall and water level, for the worst-affected rivers.